In [2]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import zuko
import lampe.inference, lampe.utils
from itertools import islice

In [3]:
mu = 42 # 'true' mu
N = 50000
prior_over_mu = zuko.distributions.Normal(torch.Tensor([0.0]), torch.Tensor([25.0]))

Utils

In [4]:
def true_data_generating_process():
    """Return x = (mean,var) of a sample of size 100 drawn from N(mu, 2)

    Returns:
        tuple: mean, var
    """
    dist = torch.distributions.Normal(mu, 2.0)
    sample = dist.sample((100, ))
    mean, var = torch.mean(sample).item(), torch.var(sample).item()
    return mean, var

Simulation (line 1 & 2 of Algorithm 1)

In [5]:
def simulator(thetas: torch.Tensor):
    """Simulator, maps theta -> x

    Args:
        thetas (torch.Tensor): Parameters

    Returns:
        torch.Tensor: Tensor of size (theta.size, 2) of (mean, var)
    """
    N = thetas.size()[0]
    x = torch.empty((N, 2))
    for i, theta in enumerate(thetas):
        dist = torch.distributions.Normal(theta, 1.0)
        samples = dist.sample((100,))
        means, var = torch.mean(samples), torch.var(samples)
        x[i][0], x[i][1] = means, var
    return x

theta = prior_over_mu.sample((N, ))
x = simulator(theta)
dataset = lampe.data.JointDataset(theta, x)

Train NPE q(theta|x) on the simulated dataset {(thetai, xi)}, i=1, ... N
Uses a neural spline flow defining the transform on the interval [-5, 5] using 10  spline segments and 5 coupling layers. The base of the flow is a standard Gaussian distribution. 

In [6]:
def build_nsf(features, context):
    return zuko.flows.NSF(features, context, bins=10, transforms=5)
q_NPE = lampe.inference.NPE(theta_dim = 1, x_dim=2, build=build_nsf)

q_NPE

NPE(
  (flow): NSF(
    (transforms): ModuleList(
      (0): MaskedAutoregressiveTransform(
        (base): MonotonicRQSTransform(bins=10)
        (order): [0]
        (hyper): MaskedMLP(
          (0): MaskedLinear(in_features=3, out_features=64, bias=True)
          (1): ReLU()
          (2): MaskedLinear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): MaskedLinear(in_features=64, out_features=29, bias=True)
        )
      )
      (1): MaskedAutoregressiveTransform(
        (base): MonotonicRQSTransform(bins=10)
        (order): [0]
        (hyper): MaskedMLP(
          (0): MaskedLinear(in_features=3, out_features=64, bias=True)
          (1): ReLU()
          (2): MaskedLinear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): MaskedLinear(in_features=64, out_features=29, bias=True)
        )
      )
      (2): MaskedAutoregressiveTransform(
        (base): MonotonicRQSTransform(bins=10)
        (order): [0]
        (h

In [18]:
#Param for training
max_epochs = 50
learning_rate_NSF = 5*10e-4
batch_size = 256
#Optim
optimizer = torch.optim.Adam(q_NPE.parameters(), learning_rate_NSF)
step = lampe.utils.GDStep(optimizer)
#Creates the loader
loader = lampe.data.JointLoader(prior_over_mu, simulator, batch_size=batch_size, vectorized=True)
loader =  DataLoader(dataset,batch_size,shuffle=True)
loader = lampe.data.DataLoader(dataset, batch_size, shuffle=True)
loss = lampe.inference.NPELoss(q_NPE)

q_NPE.train()

for epoch in range(max_epochs):
    loss_epoch = 0
    for theta, x in loader:
        losses = loss(theta, x)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        loss_epoch += losses
    #losses = torch.stack([step(loss(theta,x)) for theta,x in islice(loader, N)]).mean()
    print(f" Epoch {epoch} and loss = {loss_epoch.mean()}")
    
#TODO Mettre le earlystopping, ! prendre le même test batch
#TODO Si trop long prendre le loader de pytorch. 

 Epoch 0 and loss = 354.6964416503906
 Epoch 1 and loss = 331.89239501953125
 Epoch 2 and loss = 353.1500549316406
 Epoch 3 and loss = 308.2594909667969
 Epoch 4 and loss = 356.8833923339844
 Epoch 5 and loss = 318.5697326660156
 Epoch 6 and loss = 343.8705139160156
 Epoch 7 and loss = 301.3424377441406
 Epoch 8 and loss = 388.9856262207031
 Epoch 9 and loss = 296.6233825683594
 Epoch 10 and loss = 277.602783203125
 Epoch 11 and loss = 227.59365844726562
 Epoch 12 and loss = 282.4546203613281
 Epoch 13 and loss = 257.55694580078125
 Epoch 14 and loss = 308.4932556152344
 Epoch 15 and loss = 298.7363586425781
 Epoch 16 and loss = 377.0523681640625
 Epoch 17 and loss = 391.06402587890625
 Epoch 18 and loss = 371.72613525390625
 Epoch 19 and loss = 305.60357666015625
 Epoch 20 and loss = 280.2598571777344
 Epoch 21 and loss = 345.0688781738281
 Epoch 22 and loss = 358.43328857421875
 Epoch 23 and loss = 209.4481201171875
 Epoch 24 and loss = 336.76971435546875
 Epoch 25 and loss = 301.704

Train q(x) on {xi} i=1, ... N (so first gen the x)
For q(x) uses of a block neural autoregressive flow, single hidden layer of size 8D, x in R^d

NAF dans zuko? (mais pas "block" ... )

In [ ]:
#Preparaing dataset & loader
dataset = TensorDataset(x)
batch_size = 256
loader = DataLoader(dataset,batch_size,shuffle=True)

q_x = ??? #Pas trouvé dans le package encore. 

#Training param
max_epochs = 50
learning_rate_BNAF = 10-2
epsilon = 1e-5
#Optim
optimizer = torch.optim.Adam(q_x.parameters(), learning_rate_BNAF)
#Early stop
with torch.no_grad():
    previous_loss = -q_x.log_prob(x_test).mean()
counter=1
patience = 5
#Training

for epoch in range(max_epochs):
    print(f" Epoch = {epoch+1}")
    
    for batch_idx, batch in enumerate(loader):
        thetas_batch, x_batch = batch
        loss = -q_x.log_prob(x_test).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        loss_test = -q_x.log_prob(x_test).mean()
        if np.abs(loss_test - previous_loss) < epsilon:
            counter += 1
        if counter == patience:
            print(f"Early stop at iteration {i}")
            break
        previous_loss = loss_test
    print(f"Loss on the validation set = {previous_loss}")

SyntaxError: invalid syntax (1279069519.py, line 6)

Sample x~m from p(x | y0) % p(y0 \ x) q(x), m = 1, ... M using MCMC

Spike and slab

In [ ]:
def error_model(y, x):
    """Spike and slab model, computing p(y | x)

    Args:
        y (torch.Tensor): An observation
        x (torch.Tensor): A context sampled from q(x)

    Returns:
        float: p(y | x)
    """
    rho = 0.5
    sigma = 0.01
    tau =0.25
    res = 1.0
    D = x.size()[0]
    for j in range(D):
        dist_normal = torch.distributions.Normal(torch.Tensor([x[j]]), torch.Tensor([sigma]))
        dist_cauchy = torch.distributions.Cauchy(torch.Tensor([x[j]]), tau)
        well_specified_part = (1 - rho) * torch.exp(dist_normal.log_prob(y[j]))
        miss_specified_part = rho * torch.exp(dist_cauchy.log_prob(y[j]))
        res = res * (well_specified_part + miss_specified_part)
        return res

In [ ]:
M = 100000
warm_up_steps = 20000
traj_lenght = 1
target_acceptance_prob = 0.95

Il faut aussi faire les 2 autres simu cad NPE (comme avant) et en mettant directement le modele d'erreur dans le simu

In [ ]:
def noisy_simulator():
    #Sample the parameters theta [mu]
    sigma = 0.01
    tau = 0.25
    rho = 1/2
    X, thetas, X_test, thetas_test = simulate()
    res_train, res_test = X.clone(), X_test.clone()
    D = X[0].size()[0]
    for i, x in enumerate(X):
        spike_dist = torch.distributions.Normal(x, torch.Tensor([sigma]))
        slab_dist = torch.distributions.Cauchy(x, torch.Tensor([tau]))
        spike = ((1 - rho) *spike_dist.sample((1, )))[0]
        slab = (rho * slab_dist.sample((1, )))[0]
        res_train[i] += spike + slab
    for i, x in enumerate(X_test):
        spike_dist = torch.distributions.Normal(x, torch.Tensor([sigma]))
        slab_dist = torch.distributions.Cauchy(x, torch.Tensor([tau]))
        spike = ((1 - rho) *spike_dist.sample((1, )))[0]
        slab = (rho * slab_dist.sample((1, )))[0]
        res_test[i] += spike + slab
    return res_train, thetas, res_test, thetas_test
        

In [ ]:
X_noisy, thetas, X_noisy_test, thetas_test = noisy_simulator()

In [ ]:
q_NPE_noisy = zuko.flows.NSF(features=1, context=2, bins=10, transforms=5) #TODO! Find how to specifiy the recquired params. 

In [ ]:
#Creates the dataset & loaded
dataset = TensorDataset(thetas_vect, X_noisy)
batch_size = 256
loader = DataLoader(dataset,batch_size,shuffle=True)
#Param for training
max_epochs = 50
learning_rate_NSF = 5*10e-4
epsilon = 1e-3
#Optim
optimizer = torch.optim.Adam(q_NPE_noisy.parameters(), learning_rate_NSF)
#Preparing for ealry stop
with torch.no_grad():
    previous_loss = -q_NPE_noisy(X_noisy_test).log_prob(thetas_test).mean()
counter=1
patience = 5
losses = []
#Training
for epoch in range(max_epochs):
    print(f" Epoch = {epoch+1}")
    losses_epoch = 0
    for batch_idx, batch in enumerate(loader):
        
        thetas_batch, x_batch = batch
        loss = -q_NPE_noisy(x_batch).log_prob(thetas_batch).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses_epoch += loss.detach().item()
    losses.append(losses_epoch)
    # Checking for improvment
    with torch.no_grad():
        loss_test = -q_NPE_noisy(X_noisy_test).log_prob(thetas_test).mean()
        if np.abs(loss_test - previous_loss) < epsilon:
            counter += 1
        if counter == patience:
            print(f"Early stop at epoch {epoch}")
            break
        previous_loss = loss_test
    print(f"Loss on the validation set = {previous_loss}")

 Epoch = 1
Loss on the validation set = 312.8606872558594
 Epoch = 2
Loss on the validation set = 312.8839111328125
 Epoch = 3
Loss on the validation set = 312.7499694824219
 Epoch = 4
Loss on the validation set = 312.8526916503906
 Epoch = 5
Loss on the validation set = 312.9048767089844
 Epoch = 6
Loss on the validation set = 312.9212951660156
 Epoch = 7
Loss on the validation set = 312.7812194824219
 Epoch = 8
Loss on the validation set = 312.8135986328125
 Epoch = 9
Loss on the validation set = 312.88018798828125
 Epoch = 10
Loss on the validation set = 312.8065185546875
 Epoch = 11
Loss on the validation set = 312.91363525390625
 Epoch = 12
Loss on the validation set = 312.8385925292969
 Epoch = 13
Loss on the validation set = 312.7762145996094
 Epoch = 14
Loss on the validation set = 312.75927734375
 Epoch = 15
Loss on the validation set = 312.79779052734375
 Epoch = 16
Loss on the validation set = 312.8404541015625
 Epoch = 17
Loss on the validation set = 312.7726135253906
 Epoc